In [1]:
# %pip install transformers sentence-transformers PyPDF2 faiss-cpu torch
# %pip install bitsandbytes accelerate
# %pip install gradio

In [2]:
import torch
from dotenv import load_dotenv
import os
from huggingface_hub import login

In [3]:
# Définir l'appareil à utiliser (GPU si disponible, sinon CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
load_dotenv()
HUGGING_FACE_KEY = os.environ.get("HuggingFace_API_KEY")
login(token=HUGGING_FACE_KEY)

In [5]:
# Extraction de texte à partir des PDF
from PyPDF2 import PdfReader
import os

def extract_text_from_pdfs(pdf_folder):
    documents = []
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            reader = PdfReader(os.path.join(pdf_folder, pdf_file))
            text = " ".join([page.extract_text() for page in reader.pages])
            documents.append(text)
    return documents


In [6]:
pdf_folder = "Corpus"  # Assurez-vous de mettre vos fichiers PDF dans ce dossier
documents = extract_text_from_pdfs(pdf_folder)
print(f"Nombre de documents chargés : {len(documents)}")


Nombre de documents chargés : 17


In [7]:
# Génération d'embeddings
from sentence_transformers import SentenceTransformer

# Charger un modèle de génération d'embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Découper les documents en segments
def chunk_text(text, chunk_size=500):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = [chunk for doc in documents for chunk in chunk_text(doc)]
embeddings = embedding_model.encode(chunks)
print(f"Nombre de segments générés : {len(chunks)}")


2024-12-03 16:57:46.091120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733241466.111758  212027 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733241466.118015  212027 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 16:57:46.139780: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Nombre de segments générés : 197


In [8]:
# Indexation avec FAISS
import faiss
import numpy as np

# Créer l'index FAISS pour les embeddings
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Fonction de recherche de segments pertinents
def search(query, top_k=5):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]


In [9]:
# Test de la recherche
query = "Comment assurer mon cheval?"
results = search(query)
print(f"Résultats de recherche pour '{query}':")
for i, res in enumerate(results):
    print(f"{i+1}: {res[:200]}...")


Résultats de recherche pour 'Comment assurer mon cheval?':
1:  vétérinaire.
ASSURANCE VIE POUR VOTRE CHEVAL
L’Assurance Vie Cheval vous protège contre les risques financiers 
liés à la perte de votre cheval. 
Vous choississez parmi 3 formules
 :
La formule Basi...
2: us aussi, contre les 
accidents qui peuvent profondément modifier votre quotidien avec  
easyPROTECT Accident. LES ASSURANCES UTILES POUR VOTRE CHEVAL
ASSURANCE VIE CHEVAL
Un cheval est plus qu’un ani...
3: ASSURANCES CHEVAUXDE-CHE-FR - 08/2021   Document non contractuel à caractère publicitaireFR LES ASSURANCES UTILES POUR VOTRE CHEVAL
ASSURANCES CHEVAUX
VOTRE CHEVAL, C’EST VOTRE PASSION
Pour vivre votr...
4: une assurance équestre 
complète et flexible pour un maximum de sérénité avec l’Assurance 
Vie Cheval et l’Assurance Frais de Chirurgie. Votre cheval est ainsi 
bien protégé.
L ’assurance Responsabili...
5:  il faut faire face. Ainsi, easyPROTECT Accident vous protège contre les conséquences financières d’un acciden

In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Charger le modèle Mistral 7B depuis Hugging Face Hub
# model_name = "mistralai/Mistral-7B-v0.1"
model_name = "jpacifico/Chocolatine-3B-Instruct-DPO-v1.2"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    # load_in_4bit=True,  # Charger le modèle en 4 bits pour économiser la mémoire
    # quantization_config={"bnb_4bit_quant_type": "nf4"},  # Type de quantification
)


tokenizer_config.json:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [13]:
# Génération de réponse avec le modèle
def Response_pipe(prompt="Que font les sociétés d'assurance ?"):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs.input_ids, max_new_tokens=1024)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [14]:
Response_pipe()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"Que font les sociétés d'assurance ?\nLes sociétés d'assurance sont des entités financières qui offrent des services d'assurance aux individus et aux entreprises. Elles fonctionnent en collectant des primes des assurés, qui constituent une réserve financière. Lorsque survient une sinistre, comme une perte de biens, une invalidité ou une décès, l'assureur rembourse les assurés conformément aux termes de leur contrat d'assurance.\n\nPour garantir la stabilité financière et la capacité à honorer les engagements envers les assurés, les sociétés d'assurance suivent des règles strictes de gestion des risques et des fonds. Elles investissent également les primes collectées dans divers instruments financiers, tels que des obligations, des actions et des placements immobiliers, pour générer des revenus supplémentaires.\n\nEn résumé, les sociétés d'assurance jouent un rôle crucial dans la gestion des risques financiers, en offrant une protection aux individus et aux entreprises contre les conséq

In [15]:
# Fonction pour générer une réponse avec un contexte fourni
def generate_answer(question, context):
    input_text = f"""
Ton rôle est de répondre avec clarté à la question posée en te basant sur le éléments qui suivent: {context}

Maitenant que tu as pris connaissance du contexte, apporte un réponse précise à la question posée qui est la suivante
Question: {question}

Apporte la réponse à partie de ce point Answer:
"""

    response = Response_pipe(input_text) 
    
    return response

In [16]:
# Pipeline complet : Récupération de contexte et génération de réponse
def rag_pipeline(question):
    context = "\n".join(search(question))
    answer = generate_answer(question, context)
    return answer

In [17]:
# Interface interactive pour poser des questions
question = input("Votre question : ")
response = rag_pipeline(question)
print(f"Question: {question}\nRéponse : {response}")

Question: Comment assurer mon cheval ?
Réponse : 
Ton rôle est de répondre avec clarté à la question posée en te basant sur le éléments qui suivent:  vétérinaire.
ASSURANCE VIE POUR VOTRE CHEVAL
L’Assurance Vie Cheval vous protège contre les risques financiers 
liés à la perte de votre cheval. 
Vous choississez parmi 3 formules
 :
La formule Basique assure le décès par accident de votre cheval.
La formule Premium permet d’assurer en plus de la formule 
Basique, le décès par maladie. 
La formule Excellence est destinée aux cavaliers professionnels
afin d’assurer l’invalidité totale pour le cheval. 
ASSUREZ VOTRE REMORQUE OU VOTRE VAN DÉDIÉ
us aussi, contre les 
accidents qui peuvent profondément modifier votre quotidien avec  
easyPROTECT Accident. LES ASSURANCES UTILES POUR VOTRE CHEVAL
ASSURANCE VIE CHEVAL
Un cheval est plus qu’un animal de compagnie. C’est un ami, un 
partenaire dans le sport et au travail. 
S’il décède à la suite d’une maladie ou d’un accident ou s’il ne peut 
pl

In [21]:
from openai import OpenAI
import openai

openai.api_key = os.environ.get("OPENAI_API_KEY")

In [22]:
# Initialiser le client OpenAI
client = OpenAI()


# Fonction pour générer une réponse avec un contexte fourni
def generate_answer_gpt(question):

    context = "\n".join(search(question))

    prompt = f"""
Tu es un assistant IA chez LALUX, la compagnie d'assurance. Ton rôle est de répondre avec clarté à la question posée en te basant sur le éléments qui suivent: {context}

Maitenant que tu as pris connaissance du contexte, apporte un réponse précise à la question posée qui est la suivante
Question: {question}

Apporte la réponse à partie de ce point Answer:
"""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=512
    )    

    return response.choices[0].message.content

In [23]:
import gradio as gr

In [24]:
# Définir l'interface Gradio
interface = gr.Interface(
    fn=generate_answer_gpt,
    inputs="text",
    outputs="text",
    title="Chatbot RAG avec Chocolatine",
    description="Entrez une question et obtenez une réponse du modèle.",
)

In [25]:
# Lancer l'interface dans le notebook
interface.launch(share=False, server_port=8501)

* Running on local URL:  http://127.0.0.1:8501

To create a public link, set `share=True` in `launch()`.


In [26]:
interface.close()

Closing server running on port: 8501


In [27]:
question = input("Votre question : ")
generated_response = generate_answer_gpt(question)

In [28]:
print(generated_response )

Pour assurer votre cheval, vous pouvez souscrire à l'une des formules d'assurance proposées par LALUX. Il existe trois formules :

1. La formule Basique : elle couvre le décès de votre cheval par accident.

2. La formule Premium : en plus de la couverture de base, elle assure également en cas de décès par maladie.

3. La formule Excellence : cette formule est destinée aux cavaliers professionnels et assure l'invalidité totale pour le cheval.

De plus, vous pouvez également opter pour l'Assurance Frais de Chirurgie qui couvre les prestations hospitalières si votre cheval doit subir une intervention chirurgicale.

Il est également recommandé de souscrire à une assurance Responsabilité Civile pour se protéger des conséquences financières en cas de dommages causés par votre cheval.

Enfin, pour votre propre protection, l'assurance easyPROTECT Accident est disponible pour vous protéger contre les conséquences financières d'un accident du quotidien ou même à cheval.
